In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
_ = """
Function decorators let us “mark” functions in the source code to enhance their behavior
in some way.
"""
# decorator 101
_ = """
A decorator is a callable that takes another function as argument (the decorated function)
The decorator may perform some processing with the decorated function, and
returns it or replaces it with another function or callable object.

To summarize: the first crucial fact about decorators is that they have the power to
replace the decorated function with a different one. The second crucial fact is that they
are executed immediately when a module is loaded. This is explained next
"""
# @decorate
# def target():
#     print('running target()')
# has the same effect
# def target():
#     print('running target()')
# target = decorate(target)

def deco(func):
    def inner():
        print('running inner()')
    return inner


@deco
def target():
    print('running target()')


target()

running inner()


In [5]:
# when python executes decorators
_ = """
A key feature of decorators is that they run right after the decorated function is defined
"""
registry = []
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func
@register
def f1():
    print('running f1()')
@register
def f2():
    print('running f2()')
def f3():
    print('running f3()')
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
main()

running register(<function f1 at 0x0000025A92CD8378>)
running register(<function f2 at 0x0000025A92CD8C80>)
running main()
registry -> [<function f1 at 0x0000025A92CD8378>, <function f2 at 0x0000025A92CD8C80>]
running f1()
running f2()
running f3()


In [6]:
# decorator-enhanced strategy pattern
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order_promo(order):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)


